# **read bitcoin data**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from collections import Counter
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from sklearn.metrics import r2_score
from sklearn import metrics


def read_bit_data(dataName):
    df = pd.read_csv(dataName,thousands=',' ) #, sep='\t',
    return df


#read bitcoin data
bit_data_path="bitcoin_data.csv"
bit_data=read_bit_data(bit_data_path)

#change date column format
bit_data["Date"]=bit_data["Date"].apply(lambda date:datetime.strptime(date, '%b %d, %Y').strftime('%Y-%m-%d')) 



#preprocess data
bit_data["Price"]=pd.to_numeric(bit_data["Price"])


bit_data["Vol."]=bit_data["Vol."].apply(lambda x:x.split(sep="K")[0] )
bit_data["Vol."]=bit_data["Vol."].apply(lambda x:x.split(sep="M")[0])
bit_data.drop(bit_data[bit_data["Vol."]=="-"].index,inplace=True)
bit_data["Vol."]=bit_data["Vol."].apply(lambda x: float(x) )
bit_data["Change %"]=bit_data["Change %"].apply(lambda x:x.split(sep="%")[0] )
bit_data["Change %"]=bit_data["Change %"].apply(lambda x: float(x) )


#Split data into train and test sets
train_data_idx=bit_data.index[bit_data["Date"]=="2020-01-01"].tolist()[0]
bit_data.drop("Date",axis=1,inplace=True)

scaler = MinMaxScaler()
scaler.fit(bit_data)
bit_data_normalized=scaler.transform(np.array(bit_data))
bit_data_normalized=pd.DataFrame(data=bit_data_normalized,columns=bit_data.columns)
bit_data_label=bit_data["Price"].values
bit_data_label=np.array(bit_data_label)

bit_train_data=bit_data_normalized.iloc[train_data_idx:,:]
bit_train_label=bit_train_data["Price"].values
bit_train_data.drop("Price",axis=1,inplace=True)
bit_test_data=bit_data_normalized.iloc[:train_data_idx,:]
bit_test_label=bit_test_data["Price"].values
bit_test_data.drop("Price",axis=1,inplace=True)
bit_train_data=np.array(bit_train_data)
bit_test_data=np.array(bit_test_data)
bit_train_label=np.array(bit_train_label)
bit_test_label=np.array(bit_test_label)
print(bit_train_data.shape)
print(bit_test_data.shape)



(3449, 5)
(486, 5)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# **create CNN Model**

In [2]:
import tensorflow
from tensorflow import keras

input_layer=keras.layers.Input(shape=(bit_train_data.shape[1],1),name="input_layer")
model=keras.models.Sequential([input_layer,
                                keras.layers.Conv1D(filters=16,kernel_size=(1),padding="same",name="conv1"),
                                keras.layers.MaxPool1D(pool_size=(2),name="maxpool1"),
                                keras.layers.Conv1D(filters=8,kernel_size=(1),padding="same",name="conv2"),
                                keras.layers.Flatten(),
                                keras.layers.Dense(200,activation="relu",name="dens1"),
                                keras.layers.Dense(100,activation="relu",name="dens2"),
                                keras.layers.Dense(20,activation="relu",name="dens3"),
                                keras.layers.Dense(1,activation="sigmoid",name="out_layer")
                                
                                ]
                                )

# **compile model**

In [ ]:
model.compile(optimizer="Adam",loss=keras.losses.mean_squared_error,metrics=tensorflow.metrics.RootMeanSquaredError())
model.summary()
keras.utils.plot_model(model)

# **fit and train model**

In [ ]:
data_train_reshaped=bit_train_data.reshape((bit_train_data.shape[0],bit_train_data.shape[1],1))
model.fit(data_train_reshaped,bit_train_label,epochs=10 ,batch_size=100)


# **test model**

In [6]:

def get_acc(y_real,y_pred):
   y_real=np.array(y_real).reshape(-1)
   y_pred=np.array(y_pred).reshape(-1)
   diff=np.abs(y_real-y_pred)
   threshold=np.abs(0.1*y_real)
   true_inst=diff[diff<threshold]
   acc=(len(true_inst)/len(y_pred))*100
   return acc

print('--------test------')
data_test_reshaped=bit_test_data.reshape((bit_test_data.shape[0],bit_train_data.shape[1],1))
y_pred=model.predict(data_test_reshaped,verbose=2)
mse=metrics.mean_squared_error(bit_test_label, y_pred)
rmse=metrics.mean_squared_error(bit_test_label, y_pred,squared=False)
mae= metrics.mean_absolute_error(bit_test_label,y_pred)
r2=r2_score(bit_test_label, y_pred)
print('Mean squared error (MSE): %.3f ' % mse)  
print('Root mean square error (RMSE) : %.3f '%rmse) 
print('Mean Absolute Error (MAE): %.3f'%mae)
print('R2 score : %.2f ' %r2 )
acc=get_acc(bit_test_label, y_pred)
print("Accuracy is: %.3f" %acc)
out=np.array([round(mse,3),round(rmse,3),round(mae,3),round(r2,3),round(acc,3)])
np.savetxt("CNN_result.csv",out.reshape(1,-1) , header="mse,rmse,mae,r2,accuracy", delimiter=",")


--------test------
16/16 - 0s
Mean squared error (MSE): 0.016 
Root mean square error (RMSE) : 0.127 
Mean Absolute Error (MAE): 0.075
R2 score : 0.78 
Accuracy is: 34.568


# **create LSTM model**

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(units=100,return_sequences=True,  input_shape=(1,bit_train_data.shape[1])))
model.add(keras.layers.LSTM(units=100, return_sequences=True,activation="relu"))
model.add(keras.layers.LSTM(units=100,activation="relu"))
model.add(keras.layers.Dense(units=1 ))
model.compile(optimizer="Adam",loss=keras.losses.mean_squared_error,metrics=tensorflow.metrics.RootMeanSquaredError())
model.summary()
keras.utils.plot_model(model)




In [ ]:
data_train_reshaped=bit_train_data.reshape((bit_train_data.shape[0],1,bit_train_data.shape[1] ))

model.fit(data_train_reshaped,bit_train_label,epochs=100,)


# **test model**

In [11]:
def get_acc(y_real,y_pred):
   y_real=np.array(y_real).reshape(-1)
   y_pred=np.array(y_pred).reshape(-1)
   diff=np.abs(y_real-y_pred)
   threshold=np.abs(0.1*y_real)
   true_inst=diff[diff<threshold]
   acc=(len(true_inst)/len(y_pred))*100
   return acc

bit_test_data_reshaped=bit_test_data.reshape((bit_test_data.shape[0],1,bit_test_data.shape[1] ))
y_pred=model.predict(bit_test_data_reshaped,verbose=2)
mse=metrics.mean_squared_error(bit_test_label, y_pred)
rmse=metrics.mean_squared_error(bit_test_label, y_pred,squared=False)
mae= metrics.mean_absolute_error(bit_test_label,y_pred)
r2=r2_score(bit_test_label, y_pred)
print('Mean squared error (MSE): %.3f ' % mse)  
print('Root mean square error (RMSE) : %.3f '%rmse) 
print('Mean Absolute Error (MAE): %.3f'%mae)
print('R2 score : %.2f ' %r2 )
acc=get_acc(bit_test_label, y_pred)
print("Accuracy is: %.3f" %acc)
out=np.array([round(mse,3),round(rmse,3),round(mae,3),round(r2,3),round(acc,3)])
np.savetxt("LSTM_result.csv",out.reshape(1,-1) , header="mse,rmse,mae,r2,accuracy", delimiter=",")


16/16 - 1s
Mean squared error (MSE): 0.004 
Root mean square error (RMSE) : 0.060 
Mean Absolute Error (MAE): 0.028
R2 score : 0.95 
Accuracy is: 82.099


# **predict price variation**

In [12]:
variation=[]
# variation.append(1)
for i in range(1,len(bit_data)):
    if (bit_data.iloc[i-1,0]>bit_data.iloc[i,0]):
        variation.append(1)
    # elif (bit_data.iloc[i-1,0]==bit_data.iloc[i,0]):
    #     variation.append(0)
    else:
      variation.append(0)
variation.append(0)   
bit_data["variation"]=variation
print(bit_data)

        Price     Open     High      Low    Vol.  Change %  variation
0     57807.1  57719.1  58449.4  57029.5   63.41      0.15          1
1     57720.3  53562.3  57925.6  53088.7  103.74      7.77          1
2     53560.8  54838.6  55173.7  52400.0   83.90     -2.34          0
3     54841.4  55036.0  56419.9  53876.4   86.96     -0.35          0
4     55036.5  54011.1  55427.8  53345.0   84.08      1.88          1
...       ...      ...      ...      ...     ...       ...        ...
3936      0.1      0.1      0.1      0.1    2.16      0.00          0
3937      0.1      0.1      0.1      0.1    0.58      0.00          0
3938      0.1      0.1      0.1      0.1    0.26      0.00          0
3939      0.1      0.1      0.1      0.1    0.57      0.00          0
3940      0.1      0.0      0.1      0.1    0.08      0.00          0

[3935 rows x 7 columns]


# **predict price at next step**

In [23]:
bit_data_label=bit_data["variation"].values
bit_data_label=np.array(bit_data_label)

scaler = MinMaxScaler()
scaler.fit(bit_data)
bit_data_normalized=scaler.transform(np.array(bit_data))
bit_data_normalized=pd.DataFrame(data=bit_data_normalized,columns=bit_data.columns)
bit_data_normalized.drop("variation",inplace=True,axis=1)

bit_train_data=bit_data_normalized.iloc[train_data_idx:,:]
bit_test_data=bit_data_normalized.iloc[:train_data_idx,:]
bit_train_data=np.array(bit_train_data)
bit_test_data=np.array(bit_test_data)
bit_train_label=bit_data_label[train_data_idx:]
bit_test_label=bit_data_label[:train_data_idx]
bit_train_label=np.array(bit_train_label)
bit_test_label=np.array(bit_test_label)
print(bit_train_data.shape)
print(bit_test_data.shape)
nfeature=bit_data_normalized.shape[1]
 


(3449, 6)
(486, 6)


In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(units=100,return_sequences=True,  input_shape=(1,bit_train_data.shape[1])))
model.add(keras.layers.LSTM(units=100,activation="relu"))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=500,activation="relu"))
model.add(keras.layers.Dense(units=300,activation="relu"))
model.add(keras.layers.Dense(units=1,activation="sigmoid"))
model.compile(optimizer="adam",loss=keras.losses.binary_crossentropy,metrics=["Accuracy"])
model.summary()
keras.utils.plot_model(model)



In [ ]:
data_train_reshaped=bit_train_data.reshape((bit_train_data.shape[0],1,bit_train_data.shape[1] ))
model.fit(data_train_reshaped,bit_train_label,epochs=300)

In [35]:
data_train_reshaped[0],bit_train_label[0]

(array([[0.1133083 , 0.1132503 , 0.11206445, 0.11568051, 0.42043124,
         0.14531151]]), 1)

# **Tournement data prediction**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!gdown https://drive.google.com/u/0/uc?id=1UOqwSFd29zw8cuYb6EAf9XiM9ZGPLrqB&export=download
!mv Dataset\ 26\ 27.zip tournement_stock_data.zip

In [ ]:
!unzip /content/stock_tournement_data.zip

In [21]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import pandas as pd
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from mlxtend.plotting import plot_confusion_matrix
from collections import Counter
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

from sklearn import metrics


def read_tournement_data():
  chunk_size=20000
  batch_no=1
  for chunk in pd.read_csv('tournament_data.csv',chunksize=chunk_size,index_col="id"):
      chunk.to_csv('chunk_tournament.csv',index=False)
      batch_no+=1
      break


def read_train_data():
  chunk_size=30000
  batch_no=1
  for chunk in pd.read_csv('training_data.csv',chunksize=chunk_size,index_col="id"):
      chunk.to_csv('chunk_train.csv',index=False)
      batch_no+=1
      break
 

def get_acc(y_real,y_pred):
   y_real=np.array(y_real).reshape(-1)
   y_pred=np.array(y_pred).reshape(-1)
   diff=np.abs(y_real-y_pred)
   threshold=np.abs(0.1*y_real)
   true_inst=diff[diff<threshold]
   acc=(len(true_inst)/len(y_pred))*100
   return acc


def save_results(f_name,y_pred,test_label):
  np.savetxt(f_name+"_predictions.csv",y_pred,delimiter=",")
  mse=metrics.mean_squared_error(test_label, y_pred)
  rmse=metrics.mean_squared_error(test_label, y_pred,squared=False)
  mae= metrics.mean_absolute_error(test_label,y_pred)
  r2=metrics.r2_score(test_label, y_pred)
  print('Mean squared error (MSE): %.3f ' % mse)  
  print('Root mean square error (RMSE) : %.3f '%rmse) 
  print('Mean Absolute Error (MAE): %.3f'%mae)
  print('R2 score : %.2f ' %r2 )
  acc=get_acc(test_label, y_pred)
  print("Accuracy is: %.3f" %acc)
  out=np.array([round(mse,3),round(rmse,3),round(mae,3),round(r2,3),round(acc,3)])
  np.savetxt(f_name+"_results.csv",out.reshape(1,-1) , header="mse,rmse,mae,r2,accuracy", delimiter=",")  
 
 

#read stock data
read_train_data()
training_data=pd.read_csv("chunk_train.csv")
training_label=training_data["target"]
read_tournement_data()
tournament_data=pd.read_csv("chunk_tournament.csv")
tournament_label=tournament_data["target"]
training_label=np.array(training_label)
tournament_label=np.array(tournament_label)



training_data=training_data.drop("target",axis=1)
tournament_data=tournament_data.drop("target", axis=1)
 

#categorical features encoding
from sklearn.preprocessing import OrdinalEncoder
training_data_cat= training_data.select_dtypes(exclude=["number"])
enc = OrdinalEncoder()
enc.fit(training_data_cat)
training_data_cat=enc.transform(training_data_cat)
training_data["era"]=training_data_cat[:,0]
training_data["data_type"]=training_data_cat[:,1]

tournament_data_cat= tournament_data.select_dtypes(exclude=["number"])
enc = OrdinalEncoder()
enc.fit(tournament_data_cat)
tournament_data_cat=enc.transform(tournament_data_cat)
tournament_data["era"]=tournament_data_cat[:,0]
tournament_data["data_type"]=tournament_data_cat[:,1]


print("XGB is starting")
xgb_model=XGBRegressor(booster="gblinear",max_depth=5,learning_rate=0.01,n_estimators=2000,colsample_bytree=0.1  )
xgb_model.fit(training_data,training_label)
predictions=xgb_model.predict(tournament_data)
 
save_results("XGB",predictions,tournament_label)


print("Linear Regression is starting")
lin_reg_model=LinearRegression()
lin_reg_model.fit(training_data,training_label)
predictions=lin_reg_model.predict(tournament_data)
save_results("linear_regression",predictions,tournament_label)

print("Ridge Regression is starting")
Ridge(alpha=1.0)
lin_reg_model=LinearRegression()
lin_reg_model.fit(training_data,training_label)
predictions=lin_reg_model.predict(tournament_data)
save_results("Ridge_regression",predictions,tournament_label)
 

XGB is starting
[11:28:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean squared error (MSE): 0.050 
Root mean square error (RMSE) : 0.223 
Mean Absolute Error (MAE): 0.152
R2 score : 0.00 
Accuracy is: 50.205
Linear Regression is starting
Mean squared error (MSE): 0.051 
Root mean square error (RMSE) : 0.225 
Mean Absolute Error (MAE): 0.162
R2 score : -0.02 
Accuracy is: 44.520
Ridge Regression is starting
Mean squared error (MSE): 0.051 
Root mean square error (RMSE) : 0.225 
Mean Absolute Error (MAE): 0.162
R2 score : -0.02 
Accuracy is: 44.520


# **Enseble models**

In [ ]:
def bagging_regression_func(train_data,test_data,train_label,test_label ):
    print("------------ bagging regression is starting-------------- ")
    regr =BaggingRegressor(n_estimators=300, random_state=0)
    regr.fit(train_data, train_label)
    y_pred = regr.predict(test_data)
    mse=metrics.mean_squared_error(test_label, y_pred)
    rmse=metrics.mean_squared_error(test_label, y_pred,squared=False)
    mae= metrics.mean_absolute_error(test_label,y_pred)
    r2=r2_score(test_label, y_pred)
    print('Mean squared error (MSE): %.3f ' % mse)  
    print('Root mean square error (RMSE) : %.3f '%rmse) 
    print('Mean Absolute Error (MAE): %.3f'%mae)
    print('R2 score : %.2f ' %r2 )
    acc=get_acc(test_label, y_pred)
    print("Accuracy is: %.3f" %acc)
    out=np.array([round(mse,3),round(rmse,3),round(mae,3),round(r2,3),round(acc,3)])
    np.savetxt("tournement/bagging_regression_result_on_tournement.csv",out.reshape(1,-1) , header="mse,rmse,mae,r2,accuracy", delimiter=",")  
    return y_pred

def voting_regression_func_lin(train_data,test_data,train_label,test_label ):
    print("------------voting regression is starting-------------- ")
    r1 = LinearRegression()
    r2 = RandomForestRegressor(n_estimators=10, random_state=1)
    regr = VotingRegressor([('lr', r1), ('rf', r2)])
    regr.fit(train_data, train_label)
    y_pred = regr.predict(test_data)
    mse=metrics.mean_squared_error(test_label, y_pred)
    rmse=metrics.mean_squared_error(test_label, y_pred,squared=False)
    mae= metrics.mean_absolute_error(test_label,y_pred)
    r2=r2_score(test_label, y_pred)
    print('Mean squared error (MSE): %.3f ' % mse)  
    print('Root mean square error (RMSE) : %.3f '%rmse) 
    print('Mean Absolute Error (MAE): %.3f'%mae)
    print('R2 score : %.2f ' %r2 )
    acc=get_acc(test_label, y_pred)
    print("Accuracy is: %.3f" %acc)
    out=np.array([round(mse,3),round(rmse,3),round(mae,3),round(r2,3),round(acc,3)])
    np.savetxt("tournement/voting_with_linear_randomForest_on_tournement.csv",out.reshape(1,-1) , header="mse,rmse,mae,r2,accuracy", delimiter=",")  
    return y_pred


def voting_regression_func_mlp(train_data,test_data,train_label,test_label ):
    print("------------voting regression is starting-------------- ")
    r1=MLPRegressor(random_state=1, max_iter=300)
    r2 = RandomForestRegressor(n_estimators=10, random_state=1)
    regr = VotingRegressor([('lr', r1), ('rf', r2)])
    regr.fit(train_data, train_label)
    y_pred = regr.predict(test_data)
    mse=metrics.mean_squared_error(test_label, y_pred)
    rmse=metrics.mean_squared_error(test_label, y_pred,squared=False)
    mae= metrics.mean_absolute_error(test_label,y_pred)
    r2=r2_score(test_label, y_pred)
    print('Mean squared error (MSE): %.3f ' % mse)  
    print('Root mean square error (RMSE) : %.3f '%rmse) 
    print('Mean Absolute Error (MAE): %.3f'%mae)
    print('R2 score : %.2f ' %r2 )
    acc=get_acc(test_label, y_pred)
    print("Accuracy is: %.3f" %acc)
    out=np.array([round(mse,3),round(rmse,3),round(mae,3),round(r2,3),round(acc,3)])
    np.savetxt("tournement/voting_with_mlp_randomForest_on_tournement.csv",out.reshape(1,-1) , header="mse,rmse,mae,r2,accuracy", delimiter=",")  
    return y_pred


def boosting_regression_func_Gr(train_data,test_data,train_label,test_label ):
    print("------------boosting regression is starting-------------- ")
 
    regr =GradientBoostingRegressor(random_state=0)
    regr.fit(train_data, train_label)
    y_pred = regr.predict(test_data)
    mse=metrics.mean_squared_error(test_label, y_pred)
    rmse=metrics.mean_squared_error(test_label, y_pred,squared=False)
    mae= metrics.mean_absolute_error(test_label,y_pred)
    r2=r2_score(test_label, y_pred)
    print('Mean squared error (MSE): %.3f ' % mse)  
    print('Root mean square error (RMSE) : %.3f '%rmse) 
    print('Mean Absolute Error (MAE): %.3f'%mae)
    print('R2 score : %.2f ' %r2 )
    acc=get_acc(test_label, y_pred)
    print("Accuracy is: %.3f" %acc)
    out=np.array([round(mse,3),round(rmse,3),round(mae,3),round(r2,3),round(acc,3)])
    np.savetxt("tournement/boosting_with_GradientBoost_on_tournement.csv",out.reshape(1,-1) , header="mse,rmse,mae,r2,accuracy", delimiter=",")  
    return y_pred

def boosting_regression_func_DT(train_data,test_data,train_label,test_label ):
    print("------------boosting regression is starting-------------- ")
 
    regr =DecisionTreeRegressor(max_depth=7)
    regr.fit(train_data, train_label)
    y_pred = regr.predict(test_data)
    mse=metrics.mean_squared_error(test_label, y_pred)
    rmse=metrics.mean_squared_error(test_label, y_pred,squared=False)
    mae= metrics.mean_absolute_error(test_label,y_pred)
    r2=r2_score(test_label, y_pred)
    print('Mean squared error (MSE): %.3f ' % mse)  
    print('Root mean square error (RMSE) : %.3f '%rmse) 
    print('Mean Absolute Error (MAE): %.3f'%mae)
    print('R2 score : %.2f ' %r2 )
    acc=get_acc(test_label, y_pred)
    print("Accuracy is: %.3f" %acc)
    out=np.array([round(mse,3),round(rmse,3),round(mae,3),round(r2,3),round(acc,3)])
    np.savetxt("tournement/boosting_with_decision_tree_on_tournement.csv",out.reshape(1,-1) , header="mse,rmse,mae,r2,accuracy", delimiter=",")  
    return y_pred
bag_pred=bagging_regression_func(training_data,tournament_data,training_label,tournament_label)
vot_lin_pred=voting_regression_func_lin(training_data,tournament_data,training_label,tournament_label)
vot_mlp_pred=voting_regression_func_mlp(training_data,tournament_data,training_label,tournament_label)
boost_gr_pred=boosting_regression_func_Gr(training_data,tournament_data,training_label,tournament_label)
boost_dt_pred=boosting_regression_func_DT(training_data,tournament_data,training_label,tournament_label)

------------voting regression is starting-------------- 
Mean squared error (MSE): 0.024 
Root mean square error (RMSE) : 0.155 
Mean Absolute Error (MAE): 0.112
R2 score : 0.52 
Accuracy is: 46.700
------------voting regression is starting-------------- 
Mean squared error (MSE): 0.014 
Root mean square error (RMSE) : 0.119 
Mean Absolute Error (MAE): 0.090
R2 score : 0.72 
Accuracy is: 42.325
------------boosting regression is starting-------------- 
Mean squared error (MSE): 0.048 
Root mean square error (RMSE) : 0.220 
Mean Absolute Error (MAE): 0.153
R2 score : 0.03 
Accuracy is: 49.920
------------boosting regression is starting-------------- 
Mean squared error (MSE): 0.048 
Root mean square error (RMSE) : 0.219 
Mean Absolute Error (MAE): 0.156
R2 score : 0.04 
Accuracy is: 45.650


In [ ]:
from scipy.stats import spearmanr
def spearman_correlation(label,pred,desc):
 
  coef, p = spearmanr(label, pred)
  print(desc+'Spearmans correlation coefficient: %.3f' % coef)
  # interpret the significance
  alpha = 0.05
  if p > alpha:
    print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
  else:
    print('Samples are correlated (reject H0) p=%.3f' % p)

spearman_correlation(tournament_label,bag_pred,"Bagging ")
print("****************************************************")
spearman_correlation(tournament_label,vot_lin_pred,"voting by linear ")
print("****************************************************")
spearman_correlation(tournament_label,vot_mlp_pred,"voting by mlp  ")
print("****************************************************")
spearman_correlation(tournament_label,boost_gr_pred,"boosting by gradient descent")
print("****************************************************")
spearman_correlation(tournament_label,boost_dt_pred,"boosting by decision tree ")
 

voting by linear Spearmans correlation coefficient: 0.886
Samples are correlated (reject H0) p=0.000
voting by mlp  Spearmans correlation coefficient: 0.871
Samples are correlated (reject H0) p=0.000
boosting by gradient descentSpearmans correlation coefficient: 0.235
Samples are correlated (reject H0) p=0.000
boosting by decision tree Spearmans correlation coefficient: 0.161
Samples are correlated (reject H0) p=0.000
